In [35]:
import numpy
import random
import tensorflow as tf
import datetime
import numpy as np
import psutil
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import subprocess
import h5py
import sys

In [2]:
text = open("DATA/wiki.test.raw").read()
print("Tamaño del texto:{} \nlos primeros 100 son \n {}".format(len(text), text[:100]))
text = text[:150000]

Tamaño del texto:1290590 
los primeros 100 son 
  
 = Robert Boulter = 
 
 Robert Boulter is an English film , television and theatre actor . He had 


In [3]:
chars = sorted(list(set(text)))
char_size = len(chars)
print("El tamaño de el vocabulario es: {} y son los siguiente \n {}".format(len(chars),chars))

El tamaño de el vocabulario es: 139 y son los siguiente 
 ['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x81', '\x82', '\x85', '\x86', '\x88', '\x89', '\x8a', '\x8b', '\x8c', '\x8d', '\x8f', '\x90', '\x92', '\x93', '\x94', '\x95', '\x96', '\x98', '\x99', '\x9b', '\x9c', '\x9d', '\x9e', '\xa0', '\xa1', '\xa2', '\xa3', '\xa4', '\xa5', '\xa6', '\xa7', '\xa8', '\xa9', '\xab', '\xad', '\xaf', '\xb1', '\xb2', '\xb3', '\xb4', '\xb5', '\xbc', '\xbe', '\xc3', '\xc4', '\xc5', '\xc7', '\xe2', '\xe4', '\xe5', '\xe6', '\xe7', '\xe8', '\xe9']


In [4]:
char2id = dict((c,a) for a, c in enumerate(chars))
id2char = dict((a,c) for a, c in enumerate(chars))

In [5]:
def sample(prediction):
    r = random.uniform(0,1)
    s = 0
    
    for i in range(len(prediction)):
        s += prediction[i]
        if s >= r:
            char_id = i
            break
    
    char_one_hot = np.zeros(shape(char_size))
    char_one_hot[char_id] = 1.0
    return char_one_hot

In [6]:
seq_length = 100


dataX = []
dataY = []
for i in range(0, len(text) - seq_length, 1):
	seq_in = text[i:i + seq_length]
	seq_out = text[i + seq_length]
	dataX.append([char2id[char] for char in seq_in])
	dataY.append(char2id[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(char_size)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)


Total Patterns:  149900


In [26]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2])))
model.add(Dropout(0.25))
model.add(Dense(y.shape[1],activation = 'softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 139)               35723     
Total params: 299,915
Trainable params: 299,915
Non-trainable params: 0
_________________________________________________________________


In [28]:
#formula para la capa oculta
#Nh = Ng / (a * (Ni + No))
#Nh = Capa Oculta Ng = largo del dato Ni = Neuronas de ingreso No = neuronas de salida
model.save_weights("checkpoints/progreso-de-los-pesos-{epoch:02d}-{loss:.4f}.hdf5")

In [29]:
filepath = "checkpoints/progreso-de-los-pesos-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.save(filepath)

In [37]:
model.fit(X, y, epochs=700,batch_size=256,callbacks = callbacks_list)
subprocess.call("shutdown -s t 1 ")

Epoch 1/700
149900/149900 [==============================] - 1052s 7ms/step - loss: 3.0825 - acc: 0.1958


/home/harpie/anaconda3/envs/tensorflow/lib/python2.7/site-packages/keras/callbacks.py:402: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Epoch 2/700
149900/149900 [==============================] - 1141s 8ms/step - loss: 2.9219 - acc: 0.2155
Epoch 3/700
149900/149900 [==============================] - 1139s 8ms/step - loss: 2.8572 - acc: 0.2326
Epoch 4/700
 47616/149900 [========>.....................] - ETA: 13:49 - loss: 2.8245 - acc: 0.2388

KeyboardInterrupt: 

In [36]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([id2char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(char_size)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = id2char[index]
	seq_in = [id2char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
" e brought in to reinforce the US Army troops . In two weeks of heavy fighting , the US forces were a "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         